In [58]:
import requests
import pandas as pd
import pickle
from rossmann.Rossmann import Rossmann
import json

In [59]:
#df_sales_raw = pd.read_csv('../datasets/train.csv', low_memory=False)
df_store_raw = pd.read_csv('../datasets/store.csv', low_memory=False)
#df_sales_raw = pd.read_csv('train.csv', low_memory=False)
#df_store_raw = pd.read_csv('store.csv', low_memory=False)

# merge
#df_raw = pd.merge(df_sales_raw, df_store_raw, how='left',on='Store')

In [3]:
df_store_raw.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [60]:
# loading test dataset
df10 = pd.read_csv('/home/ezequiel/Documentos/Prejetos_Data_Science/DS_em_producao/datasets/test.csv')
# merge test dataset + store
df_test = pd.merge(df10, df_store_raw, how='left', on='Store')

# choosse store for prediction
df_test = df_test[df_test['Store'] == 47]

# remove closed days
df_test = df_test[df_test['Open'] != 0]
df_test = df_test[~df_test['Open'].isnull()]
df_test = df_test.drop('Id', axis=1)

In [36]:
df_test.head()

,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
35,47,4,2015-09-17,1.0,1,0,0,a,c,270.0,4.0,2013.0,1,14.0,2013.0,"Jan,Apr,Jul,Oct"
891,47,3,2015-09-16,1.0,1,0,0,a,c,270.0,4.0,2013.0,1,14.0,2013.0,"Jan,Apr,Jul,Oct"
1747,47,2,2015-09-15,1.0,1,0,0,a,c,270.0,4.0,2013.0,1,14.0,2013.0,"Jan,Apr,Jul,Oct"
2603,47,1,2015-09-14,1.0,1,0,0,a,c,270.0,4.0,2013.0,1,14.0,2013.0,"Jan,Apr,Jul,Oct"
4315,47,6,2015-09-12,1.0,0,0,0,a,c,270.0,4.0,2013.0,1,14.0,2013.0,"Jan,Apr,Jul,Oct"


In [61]:
model = pickle.load(open('/home/ezequiel/Documentos/Prejetos_Data_Science/DS_em_producao/model/model_rossmann.pkl', 'rb'))

In [33]:
model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [62]:
pipeline = Rossmann()

In [63]:
df1 = Rossmann.data_cleaning(pipeline, df1=df_test)

In [8]:
df1.head()

,store,day_of_week,date,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,promo_interval,month_map,is_promo
21,30,4,2015-09-17,1.0,1,0,0,a,a,40.0,2,2014,1,10,2014,"Mar,Jun,Sept,Dec",Sep,0
877,30,3,2015-09-16,1.0,1,0,0,a,a,40.0,2,2014,1,10,2014,"Mar,Jun,Sept,Dec",Sep,0
1733,30,2,2015-09-15,1.0,1,0,0,a,a,40.0,2,2014,1,10,2014,"Mar,Jun,Sept,Dec",Sep,0
2589,30,1,2015-09-14,1.0,1,0,0,a,a,40.0,2,2014,1,10,2014,"Mar,Jun,Sept,Dec",Sep,0
4301,30,6,2015-09-12,1.0,0,0,0,a,a,40.0,2,2014,1,10,2014,"Mar,Jun,Sept,Dec",Sep,0


In [64]:
df2 = Rossmann.feature_engineering(pipeline, df1)

In [10]:
df2.head()

,store,day_of_week,date,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,competition_open_since_month,...,is_promo,year,month,day,week_of_year,year_week,competition_since,competition_time_month,promo_since,promo_time_week
21,30,4,2015-09-17,1,regular_day,0,a,basic,40.0,2,...,0,2015,9,17,38,2015-37,2014-02-01,19,2014-03-03,80
877,30,3,2015-09-16,1,regular_day,0,a,basic,40.0,2,...,0,2015,9,16,38,2015-37,2014-02-01,19,2014-03-03,80
1733,30,2,2015-09-15,1,regular_day,0,a,basic,40.0,2,...,0,2015,9,15,38,2015-37,2014-02-01,19,2014-03-03,80
2589,30,1,2015-09-14,1,regular_day,0,a,basic,40.0,2,...,0,2015,9,14,38,2015-37,2014-02-01,19,2014-03-03,80
4301,30,6,2015-09-12,0,regular_day,0,a,basic,40.0,2,...,0,2015,9,12,37,2015-36,2014-02-01,19,2014-03-03,79


In [65]:
df3 = Rossmann.data_preparation(pipeline, df2)

In [12]:
df3.head()

,store,promo,store_type,assortment,competition_distance,competition_open_since_month,competition_open_since_year,promo2,promo2_since_week,promo2_since_year,competition_time_month,promo_time_week,day_of_week_sin,day_of_week_cos,month_sin,month_cos,day_sin,day_cos,week_of_year_sin,week_of_year_cos
21,30,1,0,1,0.0,2,2014,1,10,2014,0.0,1.000000,-0.433884,-0.900969,-1.0,-1.836970e-16,-4.067366e-01,-0.913545,-0.992709,-0.120537
877,30,1,0,1,0.0,2,2014,1,10,2014,0.0,1.000000,0.433884,-0.900969,-1.0,-1.836970e-16,-2.079117e-01,-0.978148,-0.992709,-0.120537
1733,30,1,0,1,0.0,2,2014,1,10,2014,0.0,1.000000,0.974928,-0.222521,-1.0,-1.836970e-16,1.224647e-16,-1.000000,-0.992709,-0.120537
2589,30,1,0,1,0.0,2,2014,1,10,2014,0.0,1.000000,0.781831,0.623490,-1.0,-1.836970e-16,2.079117e-01,-0.978148,-0.992709,-0.120537
4301,30,0,0,1,0.0,2,2014,1,10,2014,0.0,0.857143,-0.781831,0.623490,-1.0,-1.836970e-16,5.877853e-01,-0.809017,-0.970942,-0.239316


In [66]:
df_response = Rossmann.get_prediction(pipeline, model=model, original_data=df_test, test_data=df3)

In [69]:
pd.DataFrame(json.loads(df_response)).head()

,store,day_of_week,date,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,...,year,month,day,week_of_year,year_week,competition_since,competition_time_month,promo_since,promo_time_week,prediction
0,47,4,2015-09-17T00:00:00.000Z,1.0,1,regular_day,0,a,extended,270.0,...,2015,9,17,38,2015-37,2013-04-01T00:00:00.000Z,29,2013-04-01T00:00:00.000Z,128,6883.714844
1,47,3,2015-09-16T00:00:00.000Z,1.0,1,regular_day,0,a,extended,270.0,...,2015,9,16,38,2015-37,2013-04-01T00:00:00.000Z,29,2013-04-01T00:00:00.000Z,128,6964.690430
2,47,2,2015-09-15T00:00:00.000Z,1.0,1,regular_day,0,a,extended,270.0,...,2015,9,15,38,2015-37,2013-04-01T00:00:00.000Z,29,2013-04-01T00:00:00.000Z,128,7663.532715
3,47,1,2015-09-14T00:00:00.000Z,1.0,1,regular_day,0,a,extended,270.0,...,2015,9,14,38,2015-37,2013-04-01T00:00:00.000Z,29,2013-04-01T00:00:00.000Z,128,9048.933594
4,47,6,2015-09-12T00:00:00.000Z,1.0,0,regular_day,0,a,extended,270.0,...,2015,9,12,37,2015-36,2013-04-01T00:00:00.000Z,29,2013-04-01T00:00:00.000Z,127,5154.006836


# API Tester

In [2]:
import requests
import pandas as pd
import json

In [3]:
df_store_raw = pd.read_csv('../datasets/store.csv', low_memory=False)

# loading test dataset
df = pd.read_csv('/home/ezequiel/Documentos/Prejetos_Data_Science/DS_em_producao/datasets/test.csv')

In [4]:
# merge test dataset + store
df_test = pd.merge(df, df_store_raw, how='left', on='Store')

# choosse store for prediction
df_test = df_test[df_test['Store'] == 30]

# remove closed days
df_test = df_test[df_test['Open'] != 0]
df_test = df_test[~df_test['Open'].isnull()]
df_test = df_test.drop('Id', axis=1)

In [150]:
df_test.head()

,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
21,30,4,2015-09-17,1.0,1,0,0,a,a,40.0,2.0,2014.0,1,10.0,2014.0,"Mar,Jun,Sept,Dec"
877,30,3,2015-09-16,1.0,1,0,0,a,a,40.0,2.0,2014.0,1,10.0,2014.0,"Mar,Jun,Sept,Dec"
1733,30,2,2015-09-15,1.0,1,0,0,a,a,40.0,2.0,2014.0,1,10.0,2014.0,"Mar,Jun,Sept,Dec"
2589,30,1,2015-09-14,1.0,1,0,0,a,a,40.0,2.0,2014.0,1,10.0,2014.0,"Mar,Jun,Sept,Dec"
4301,30,6,2015-09-12,1.0,0,0,0,a,a,40.0,2.0,2014.0,1,10.0,2014.0,"Mar,Jun,Sept,Dec"


In [6]:
# convert dataframe to json
#data = json.dumps(df_test.to_dict(orient='records'))
data = df_test.to_json(orient='records', date_format='iso')

In [ ]:
pd.json_normalize(json.loads(data))

In [136]:
data = [{"Store": 30,
        "DayOfWeek": 4,
        "Date": "2015-09-17",
        "StoreType": "a"},
        {"Store": 30,
        "DayOfWeek": 5,
        "Date": "2015-09-16",
        "StoreType": "b"}]

In [137]:
data1 = pd.DataFrame(data=data)

In [138]:
data1

,Store,DayOfWeek,Date,StoreType
0,30,4,2015-09-17,a
1,30,5,2015-09-16,b


In [145]:
#data = data1.to_json(orient='records')
data = json.dumps(data1.to_dict(orient='records'))

In [11]:
# API call
url = 'http://localhost:5000/rossmann/predict'
#header = {'Content-type': 'application/jason'}
data = data

#r = requests.post(url, data=data, headers=header)
r = requests.post(url=url, json=data)
print(f'Status Code {r.status_code}')

Status Code 200


In [9]:
pd.json_normalize(r.json()).head()

,store,day_of_week,date,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,...,year,month,day,week_of_year,year_week,competition_since,competition_time_month,promo_since,promo_time_week,prediction
0,30,4,2015-09-17T00:00:00.000Z,1.0,1,regular_day,0,a,basic,40.0,...,2015,9,17,38,2015-37,2014-02-01T00:00:00.000Z,19,2014-03-03T00:00:00.000Z,80,5651.043457
1,30,3,2015-09-16T00:00:00.000Z,1.0,1,regular_day,0,a,basic,40.0,...,2015,9,16,38,2015-37,2014-02-01T00:00:00.000Z,19,2014-03-03T00:00:00.000Z,80,5717.520508
2,30,2,2015-09-15T00:00:00.000Z,1.0,1,regular_day,0,a,basic,40.0,...,2015,9,15,38,2015-37,2014-02-01T00:00:00.000Z,19,2014-03-03T00:00:00.000Z,80,6823.190430
3,30,1,2015-09-14T00:00:00.000Z,1.0,1,regular_day,0,a,basic,40.0,...,2015,9,14,38,2015-37,2014-02-01T00:00:00.000Z,19,2014-03-03T00:00:00.000Z,80,7919.496094
4,30,6,2015-09-12T00:00:00.000Z,1.0,0,regular_day,0,a,basic,40.0,...,2015,9,12,37,2015-36,2014-02-01T00:00:00.000Z,19,2014-03-03T00:00:00.000Z,79,3951.865479


In [80]:
url = 'http://localhost:5000/rossmann/ola'

r = requests.get(url)

In [81]:
print(r.json())

hello_world
